In [1]:
!if not exist "./files" mkdir files
# Download Face detection XML 
!curl -L -o ./files/haarcascade_frontalface_default.xml https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml
# Download emotion trained data
!curl -L -o ./files/emotion_model.hdf5 https://mechasolution.vn/source/blog/AI-tutorial/Emotion_Recognition/emotion_model.hdf5

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0  908k    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  908k  100  908k    0     0  1403k      0 --:--:-- --:--:-- --:--:-- 1401k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
 14  878k   14  124k    0     0  79959      0  0:00:11  0:00:01  0:00:10 79909
100  878k  100  878k    0     0   366k      0  0:00:02  0:00:02 --:--:--  366k


In [7]:
import cv2
import numpy as np   
import time
from keras.preprocessing.image import img_to_array
from keras.models import load_model
from collections import Counter
import heartpy as hp
from matplotlib import pyplot as plt

import urllib.request
from bs4 import BeautifulSoup
import cv2
from PIL import ImageFont, ImageDraw, Image

#전체 무한 루프문
while True:
    #루프를 탈출하지 않는 한 Main창을 계속 띄움
    image = cv2.imread("Main.jpg", cv2.IMREAD_ANYCOLOR)
    cv2.imshow("Main", image)
    
    #FaceDetectionService 함수
    def FDS():
        #FDS 시작시 Main창 닫기
        cv2.destroyAllWindows()
        
        image = cv2.imread("Main.jpg", cv2.IMREAD_ANYCOLOR)
        count1 = 0
        fp = open("data.csv","w+t")
        cntdata =[]
        start = time.time()
        print(start)

        # Face detection XML load and trained model loading
        face_detection = cv2.CascadeClassifier('files/haarcascade_frontalface_default.xml')
        emotion_classifier = load_model('files/emotion_model.hdf5', compile=False)
        EMOTIONS = ["Angry" ,"Disgusting","Fearful", "Happy", "Sad", "Surpring", "Neutral"]

        # Video capture using webcam
        camera = cv2.VideoCapture(0)

        #Face Detection 과정 루프문
        while True:
            #10초마다 기존에 있던 누적된 데이터를 기반으로 감정에 따른 메세지들
            interval = start - (time.time())
            if((int(interval) % 10 == 0 ) & (count1 > 0)):
                c = Counter(cntdata) 
                mode = c.most_common(1) 
                current_Emotion=EMOTIONS[mode[0][0]]
                if(mode[0][0] == 0):
                    image = cv2.imread("Angry.jpg", cv2.IMREAD_ANYCOLOR)
                    cv2.imshow("Moon", image)
                elif(mode[0][0] == 1):
                    image = cv2.imread("Disgust.jpg", cv2.IMREAD_ANYCOLOR)
                    cv2.imshow("Moon", image)
                elif(mode[0][0] == 2):
                    image = cv2.imread("Fear.jpg", cv2.IMREAD_ANYCOLOR)
                    cv2.imshow("Moon", image)
                elif(mode[0][0] == 4):
                    image = cv2.imread("Sad.jpg", cv2.IMREAD_ANYCOLOR)
                    cv2.imshow("Moon", image)
                del cntdata[:]



            # Capture image from camera


            ret, frame = camera.read()

            # Convert color to gray scale
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # Face detection in frame
            faces = face_detection.detectMultiScale(gray,
                                                    scaleFactor=1.1,
                                                    minNeighbors=5,
                                                    minSize=(30,30))

            # Create empty image
            canvas = np.zeros((250, 300, 3), dtype="uint8")

            # Perform emotion recognition only when face is detected
            if len(faces) > 0:
                # For the largest image
                face = sorted(faces, reverse=True, key=lambda x: (x[2] - x[0]) * (x[3] - x[1]))[0]
                (fX, fY, fW, fH) = face
                # Resize the image to 48x48 for neural network
                roi = gray[fY:fY + fH, fX:fX + fW]
                roi = cv2.resize(roi, (48, 48))
                roi = roi.astype("float") / 255.0
                roi = img_to_array(roi)
                roi = np.expand_dims(roi, axis=0)


                # Emotion predict
                preds = emotion_classifier.predict(roi)[0] ##퍼센티지 환산전 데이터
                emotion_probability = np.max(preds)
                label = EMOTIONS[preds.argmax()]


                # Assign labeling
                cv2.putText(frame, label, (fX, fY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2) ## 가장 많은 퍼센티지의 감정 표시
                cv2.rectangle(frame, (fX, fY), (fX + fW, fY + fH), (0, 0, 255), 2) ## 얼굴 인식범위 사각형 표시

                # Label printing
                for (i, (emotion, prob)) in enumerate(zip(EMOTIONS, preds)):
                    text = "{}: {:.2f}%".format(emotion, prob * 100) ## 표시할 텍스트    
                    w = int(prob * 300) ## 게이지 넓이
                    cv2.rectangle(canvas, (7, (i * 35) + 5), (w, (i * 35) + 35), (0, 0, 255), -1) ##게이지 표시
                    cv2.putText(canvas, text, (10, (i * 35) + 23), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255, 255, 255), 2) ## 감정과 수치 표시

            # Open two windows
            ## Display image ("Emotion Recognition")
            ## Display probabilities of emotion
            cv2.imshow("Emotion Recognition", frame)
            cv2.imshow("Probabilities", canvas)



            # q to quit
            if cv2.waitKey(1) & 0xFF == ord('q'):
                fp.close()
                break
            if len(faces) > 0:    
                fp.write("%d\t%d\n" %(count1, preds.argmax()))
                cntdata.append(preds.argmax())
                count1 += 1

        # Clear program and close windows
        camera.release()
        cv2.destroyAllWindows()
      
    #EventGuideService 함수
    def EGS():
        url = 'http://www.seoulnoin.or.kr/story/schedule.asp?currentDate=2019-11-11'
        html = urllib.request.urlopen(url).read()
        soup = BeautifulSoup(html,'html.parser')
        text =""
        ntitle =[]
        nurl = []

        title = soup.select('td')
        #print(soup.select(".black"))
        for i in title:
            if(i.select(".cal_link2")):
                ntitle.append((i.get_text()).strip('\n'))
                for o in i.div:
                    nurl.append(o.get('href'))
            else: 
                continue
        
        for i in range(0,len(ntitle)):
            print(ntitle[i], nurl[i])
    
        img = np.full(shape=(1080,1920,3),fill_value=255,dtype=np.uint8) 
        img = Image.fromarray(img) #img배열을 PIL이 처리가능하게 변환 

        draw = ImageDraw.Draw(img) 
        font=ImageFont.truetype("fonts/gulim.ttc",20) 
        org=(50,100) 
        for i in range(0,len(ntitle)):
            text = text + ntitle[i] + "\n" + nurl[i] + "\n" +"\n"
        text = text + "행사 수만큼 enter를 눌러주세요!"

    
        draw.text(org,text,font=font,fill=(0,0,0)) #text를 출력 
        img = np.array(img) #다시 OpenCV가 처리가능하게 np 배열로 변환 

        #OpenCV기준으로 text크기를 구해 사각형 생성 
        font=cv2.FONT_HERSHEY_SIMPLEX 
        size, BaseLine=cv2.getTextSize(text,font,1,2) 
 

        cv2.imshow("A",img) 
        for i in range(0,len(ntitle)):
            cv2.waitKey() 
        cv2.destroyAllWindows()

    def PMS():
        ppi = []
        file_path ='C:/MAVE_RawData/2020-12-14_오후 5_24/Rawdata.txt'
        ppg= np.genfromtxt(file_path , encoding='cp949', skip_header=1,usecols=(4))
        #ppg =hp.get_data(file_path)
        plt.plot(ppg)
        plt.show()

        wd, m = hp.process(ppg,256)
        print(len(wd['peaklist']))
        for i in range(0, (len(wd['peaklist']) - 1)):
            ppi.append(abs(wd['peaklist'][i] - wd['peaklist'][i+1]))
        print(wd['peaklist'])
        print(ppi)
        print("bpm: %f"%(m['bpm']))
        image = cv2.imread("test.jpg", cv2.IMREAD_ANYCOLOR)
        cv2.imshow("Moon", image)
        cv2.waitKey(5000)
        cv2.destroyAllWindows()
        ubpm = int(m['bpm'])
        if (ubpm < 60):
            image = cv2.imread("low.jpg", cv2.IMREAD_ANYCOLOR)
            cv2.imshow("Moon", image)
        elif (59 < ubpm & ubpm < 101):
            image = cv2.imread("good.jpg", cv2.IMREAD_ANYCOLOR)
            cv2.imshow("Moon", image)
        elif (ubpm > 100):
            image = cv2.imread("fast.jpg", cv2.IMREAD_ANYCOLOR)
            cv2.imshow("Moon", image)
        while True:
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        cv2.destroyAllWindows()
        plt.plot(ppi)
        plt.show()
        
    def TV():
        image = cv2.imread("TV.jpg", cv2.IMREAD_ANYCOLOR)
        cv2.imshow("Moon", image)
        while True:
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        cv2.destroyAllWindows()
        
    def Call():
        image = cv2.imread("Call.jpg", cv2.IMREAD_ANYCOLOR)
        cv2.imshow("Moon", image)
        while True:
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        cv2.destroyAllWindows()
            
    mainKey = cv2.waitKey(0)
    if(mainKey == 70 or mainKey == 102):
        FDS()
    elif(mainKey == 69 or mainKey ==101):
        EGS()
    elif(mainKey == 80 or mainKey == 112):
        PMS()
    elif(mainKey == 84 or mainKey == 116):
        TV()
    elif(mainKey == 67 or mainKey == 99):
        Call()
    elif(mainKey == 90 or mainKey ==122):
        break
    cv2.destroyAllWindows()
cv2.destroyAllWindows()